### 1 - Import test data

In [27]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2022-11-7", end="2022-12-25", interval='15m')
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-11-07 00:00:00,0.993443,0.994827,0.993443,0.994332,0.994332,0
2022-11-07 00:15:00,0.994431,0.994431,0.993345,0.993345,0.993345,0
2022-11-07 00:30:00,0.993246,0.994036,0.993147,0.993838,0.993838,0
2022-11-07 00:45:00,0.993838,0.994036,0.993147,0.993542,0.993542,0
2022-11-07 01:00:00,0.993443,0.993641,0.992851,0.992851,0.992851,0
...,...,...,...,...,...,...
2022-12-23 21:15:00,1.062248,1.062248,1.061909,1.062022,1.062022,0
2022-12-23 21:30:00,1.061909,1.062248,1.061684,1.061797,1.061797,0
2022-12-23 21:45:00,1.061797,1.062135,1.061797,1.061909,1.061909,0


### 2 - Define your signal function

In [28]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [29]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

0    3027
1     146
2     145
Name: signal, dtype: int64

### 3 - Connect to the market and execute trades

In [30]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [31]:

def get_candles(n):
    access_token="7e8d7bc933c108def6dceb5c50079014-084a221cb5d0b2adb4c70b4a51f7d5de"
    accountID = "101-001-8028197-001"
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


True
True
True


In [32]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    accountID = "101-001-8028197-001" 
    client = API("7e8d7bc933c108def6dceb5c50079014-084a221cb5d0b2adb4c70b4a51f7d5de")
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [33]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

      Open    Close     High      Low
0  1.07029  1.07036  1.07040  1.06994
1  1.07035  1.07016  1.07038  1.07004
1.07086    1.06984    1.0695    1.07052
{'orderCreateTransaction': {'id': '1826', 'accountID': '101-001-8028197-001', 'userID': 8028197, 'batchID': '1826', 'requestID': '25029157882975358', 'time': '2022-12-31T01:09:22.456512684Z', 'type': 'MARKET_ORDER', 'instrument': 'EUR_USD', 'units': '1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'takeProfitOnFill': {'price': '1.07086', 'timeInForce': 'GTC'}, 'stopLossOnFill': {'price': '1.06984', 'timeInForce': 'GTC', 'triggerMode': 'TOP_OF_BOOK'}, 'reason': 'CLIENT_ORDER'}, 'orderCancelTransaction': {'id': '1827', 'accountID': '101-001-8028197-001', 'userID': 8028197, 'batchID': '1826', 'requestID': '25029157882975358', 'time': '2022-12-31T01:09:22.456512684Z', 'type': 'ORDER_CANCEL', 'orderID': '1826', 'reason': 'MARKET_HALTED'}, 'relatedTransactionIDs': ['1826', '1827'], 'lastTransactionID': '1827'}
